In [6]:
pip install skl2onnx

  Using cached skl2onnx-1.8.0-py2.py3-none-any.whl (230 kB)
     |████████████████████████████████| 12.0 MB 6.6 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install onnxruntime

     |████████████████████████████████| 5.0 MB 3.1 MB/s 
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np 
import pandas as pd 


In [9]:
data = pd.read_csv('../../data/cleaned_cuisine.csv')
data.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
X = data.iloc[:,2:]
X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
y = data[['cuisine']]
y.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [14]:
# 5 types of model fitting
model_gaussian = GaussianNB()
model_gaussian.fit(X_train,y_train.values.ravel())
model_rfst = RandomForestClassifier()
model_rfst.fit(X_train,y_train.values.ravel())
model_nba = BernoulliNB(binarize = .5)
model_nba.fit(X_train,y_train.values.ravel())
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train,y_train.values.ravel())
model_kn = KNeighborsClassifier()
model_kn.fit(X_train,y_train.values.ravel())


KNeighborsClassifier()

In [15]:
y_pred = model_kn.predict(X_test)

In [16]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.59      0.74      0.66       239
      indian       0.83      0.80      0.82       237
    japanese       0.68      0.77      0.72       252
      korean       0.93      0.65      0.76       220
        thai       0.77      0.71      0.74       251

    accuracy                           0.73      1199
   macro avg       0.76      0.73      0.74      1199
weighted avg       0.76      0.73      0.74      1199



In [17]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
options = {id(model_kn): {'nocl': True, 'zipmap': False}}
onx = convert_sklearn(model_kn, initial_types=initial_type,options=options)
with open("./model-kn.onnx", "wb") as f:
    f.write(onx.SerializeToString())


